# Building a Lead Scoring Model with EvalML

In this demo, we will build an optimized lead scoring model using EvalML. To optimize the pipeline, we will set up an objective function to maximize the revenue generated with true positives while taking into account the cost of false positives. At the end of this demo, we also show you how introducing the right objective during the training is over 6x better than using a generic machine learning metric like AUC.

In [1]:
import evalml
from evalml import AutoMLSearch
from evalml.objectives import LeadScoring

2020-11-24 19:49:01,985 featuretools - WARNING    Featuretools failed to load plugin nlp_primitives from library nlp_primitives. For a full stack trace, set logging to debug.


## Configure LeadScoring 

To optimize the pipelines toward the specific business needs of this model, you can set your own assumptions for how much value is gained through true positives and the cost associated with false positives. These parameters are

* `true_positive` - dollar amount to be gained with a successful lead
* `false_positive` - dollar amount to be lost with an unsuccessful lead

Using these parameters, EvalML builds a pileline that will maximize the amount of revenue per lead generated.

In [2]:
lead_scoring_objective = LeadScoring(
    true_positives=1000,
    false_positives=-10
)

## Dataset

We will be utilizing a dataset detailing a customer's job, country, state, zip, online action, the dollar amount of that action and whether they were a successful lead.

In [3]:
from urllib.request import urlopen
import pandas as pd
import woodwork as ww
customers_data = urlopen('https://featurelabs-static.s3.amazonaws.com/lead_scoring_ml_apps/customers.csv')
interactions_data = urlopen('https://featurelabs-static.s3.amazonaws.com/lead_scoring_ml_apps/interactions.csv')
leads_data = urlopen('https://featurelabs-static.s3.amazonaws.com/lead_scoring_ml_apps/previous_leads.csv')
customers = pd.read_csv(customers_data)
interactions = pd.read_csv(interactions_data)
leads = pd.read_csv(leads_data)

X = customers.merge(interactions, on='customer_id').merge(leads, on='customer_id')
y = X['label']
X = X.drop(['customer_id', 'date_registered', 'birthday','phone', 'email',
        'owner', 'company', 'id', 'time_x',
        'session', 'referrer', 'time_y', 'label', 'country'], axis=1)
display(X.head(5))

,job,state,zip,action,amount
0,"Engineer, mining",NY,60091.0,page_view,NaN
1,"Psychologist, forensic",CA,NaN,purchase,135.23
2,"Psychologist, forensic",CA,NaN,page_view,NaN
3,Air cabin crew,NaN,60091.0,download,NaN
4,Air cabin crew,NaN,60091.0,page_view,NaN


We will convert our data into Woodwork data structures. Doing so enables us to have more control over the types passed to and inferred by AutoMLSearch.

In [4]:
X = ww.DataTable(X, semantic_tags={'job': 'category'}, logical_types={'job': 'Categorical'})
y = ww.DataColumn(y)
X.types

,Physical Type,Logical Type,Semantic Tag(s)
Data Column,,,
job,category,Categorical,{category}
state,category,Categorical,{category}
zip,float64,Double,{numeric}
action,category,Categorical,{category}
amount,float64,Double,{numeric}


## Search for the best pipeline

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set.

EvalML natively supports one-hot encoding and imputation so the above `NaN` and categorical values will be taken care of.

In [5]:
X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X, y, test_size=0.2, random_state=0)

print(X.types)

            Physical Type Logical Type Semantic Tag(s)
Data Column                                           
job              category  Categorical      {category}
state            category  Categorical      {category}
zip               float64       Double       {numeric}
action           category  Categorical      {category}
amount            float64       Double       {numeric}


Because the lead scoring labels are binary, we will use `AutoMLSearch(problem_type='binary')`. When we call `.search()`, the search for the best pipeline will begin. 

In [6]:
automl = AutoMLSearch(problem_type='binary',
                      objective=lead_scoring_objective,
                      additional_objectives=['auc'],
                      max_iterations=5,
                      optimize_thresholds=True)

automl.search(X_train, y_train)

{'message': 'The following labels fall below 10% of the target: [True]', 'data_check_name': 'ClassImbalanceDataCheck', 'level': 'warning', 'code': 'CLASS_IMBALANCE_BELOW_THRESHOLD', 'details': {'target_values': [True]}}
Generating pipelines to search over...
*****************************
* Beginning pipeline search *
*****************************

Optimizing for Lead Scoring. 
Greater score is better.

Searching up to 5 pipelines. 
Allowed model families: extra_trees, random_forest, decision_tree, linear_model, xgboost, lightgbm, catboost



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Batch 1: (1/5) Mode Baseline Binary Classification P... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Lead Scoring: 0.000
2/5) Decision Tree Classifier w/ Imputer +... Elapsed:00:01
	Starting cross validation
	Finished cross validation - mean Lead Scoring: 40.075
3/5) LightGBM Classifier w/ Imputer + One ... Elapsed:00:02
	Starting cross validation
	Finished cross validation - mean Lead Scoring: 39.155
4/5) Extra Trees Classifier w/ Imputer + O... Elapsed:00:06
	Starting cross validation
	Finished cross validation - mean Lead Scoring: 38.373
5/5) Elastic Net Classifier w/ Imputer + O... Elapsed:00:09
	Starting cross validation
	Finished cross validation - mean Lead Scoring: 42.140

Search finished after 00:10            
Best pipeline: Elastic Net Classifier w/ Imputer + One Hot Encoder + Standard Scaler
Best pipeline Lead Scoring: 42.140250


### View rankings and select pipeline

Once the fitting process is done, we can see all of the pipelines that were searched, ranked by their score on the lead scoring objective we defined.

In [7]:
automl.rankings

,id,pipeline_name,score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,4,Elastic Net Classifier w/ Imputer + One Hot En...,42.140250,42.129032,NaN,False,{'Imputer': {'categorical_impute_strategy': 'm...
1,1,Decision Tree Classifier w/ Imputer + One Hot ...,40.074770,42.077419,NaN,False,{'Imputer': {'categorical_impute_strategy': 'm...
2,2,LightGBM Classifier w/ Imputer + One Hot Encoder,39.154826,35.367742,NaN,False,{'Imputer': {'categorical_impute_strategy': 'm...
3,3,Extra Trees Classifier w/ Imputer + One Hot En...,38.372643,32.329032,NaN,False,{'Imputer': {'categorical_impute_strategy': 'm...
4,0,Mode Baseline Binary Classification Pipeline,0.000000,0.000000,NaN,False,{'Baseline Classifier': {'strategy': 'mode'}}


to select the best pipeline we can run

In [8]:
best_pipeline = automl.best_pipeline

### Describe pipeline

You can get more details about any pipeline, including how it performed on other objective functions by calling `.describe_pipeline()` and specifying the `id` of the pipeline.

In [9]:
automl.describe_pipeline(automl.rankings.iloc[0]["id"])

*************************************************************************
* Elastic Net Classifier w/ Imputer + One Hot Encoder + Standard Scaler *
*************************************************************************

Problem Type: binary
Model Family: Linear

Pipeline Steps
1. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : mean
	 * categorical_fill_value : None
	 * numeric_fill_value : None
2. One Hot Encoder
	 * top_n : 10
	 * features_to_encode : None
	 * categories : None
	 * drop : None
	 * handle_unknown : ignore
	 * handle_missing : error
3. Standard Scaler
4. Elastic Net Classifier
	 * alpha : 0.5
	 * l1_ratio : 0.5
	 * n_jobs : -1
	 * max_iter : 1000
	 * penalty : elasticnet
	 * loss : log

Training
Training for binary problems.
Objective to optimize binary classification pipeline thresholds for: <evalml.objectives.lead_scoring.LeadScoring object at 0x13f841610>
Total training time (including CV): 1.3 seconds

Cross Validation
------

## Evaluate on hold out

Finally, we retrain the best pipeline on all of the training data and evaluate it on the holdout dataset.

In [10]:
best_pipeline.fit(X_train, y_train)

GeneratedPipeline(parameters={'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'One Hot Encoder':{'top_n': 10, 'features_to_encode': None, 'categories': None, 'drop': None, 'handle_unknown': 'ignore', 'handle_missing': 'error'}, 'Elastic Net Classifier':{'alpha': 0.5, 'l1_ratio': 0.5, 'n_jobs': -1, 'max_iter': 1000, 'penalty': 'elasticnet', 'loss': 'log'},})

Now, we can score the pipeline on the holdout data using both the lead scoring score and the AUC.

In [11]:
best_pipeline.score(X_holdout, y_holdout, objectives=["auc", lead_scoring_objective])

OrderedDict([('AUC', 0.5), ('Lead Scoring', 0.0)])

## Why optimize for a problem-specific objective?

To demonstrate the importance of optimizing for the right objective, let's search for another pipeline using AUC, a common machine learning metric. After that, we will score the holdout data using the lead scoring objective to see how the best pipelines compare.

In [12]:
automl_auc = evalml.AutoMLSearch(problem_type='binary',
                                 objective='auc',
                                 additional_objectives=[],
                                 max_iterations=5,
                                 optimize_thresholds=True)

automl_auc.search(X_train, y_train)

{'message': 'The following labels fall below 10% of the target: [True]', 'data_check_name': 'ClassImbalanceDataCheck', 'level': 'warning', 'code': 'CLASS_IMBALANCE_BELOW_THRESHOLD', 'details': {'target_values': [True]}}
Generating pipelines to search over...
*****************************
* Beginning pipeline search *
*****************************

Optimizing for AUC. 
Greater score is better.

Searching up to 5 pipelines. 
Allowed model families: extra_trees, random_forest, decision_tree, linear_model, xgboost, lightgbm, catboost



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Batch 1: (1/5) Mode Baseline Binary Classification P... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean AUC: 0.500
2/5) Decision Tree Classifier w/ Imputer +... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean AUC: 0.600
3/5) LightGBM Classifier w/ Imputer + One ... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean AUC: 0.677
4/5) Extra Trees Classifier w/ Imputer + O... Elapsed:00:01
	Starting cross validation
	Finished cross validation - mean AUC: 0.707
5/5) Elastic Net Classifier w/ Imputer + O... Elapsed:00:02
	Starting cross validation
	Finished cross validation - mean AUC: 0.500

Search finished after 00:02            
Best pipeline: Extra Trees Classifier w/ Imputer + One Hot Encoder
Best pipeline AUC: 0.707141


In [13]:
automl_auc.rankings

,id,pipeline_name,score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,3,Extra Trees Classifier w/ Imputer + One Hot En...,0.707141,0.672147,41.428152,False,{'Imputer': {'categorical_impute_strategy': 'm...
1,2,LightGBM Classifier w/ Imputer + One Hot Encoder,0.676588,0.677781,35.317591,False,{'Imputer': {'categorical_impute_strategy': 'm...
2,1,Decision Tree Classifier w/ Imputer + One Hot ...,0.600027,0.609065,20.005390,False,{'Imputer': {'categorical_impute_strategy': 'm...
3,0,Mode Baseline Binary Classification Pipeline,0.500000,0.500000,0.000000,False,{'Baseline Classifier': {'strategy': 'mode'}}
4,4,Elastic Net Classifier w/ Imputer + One Hot En...,0.500000,0.500000,0.000000,False,{'Imputer': {'categorical_impute_strategy': 'm...


Like before, we can look at the rankings and pick the best pipeline.

In [14]:
best_pipeline_auc = automl_auc.best_pipeline

# train on the full training data
best_pipeline_auc.fit(X_train, y_train)

GeneratedPipeline(parameters={'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'One Hot Encoder':{'top_n': 10, 'features_to_encode': None, 'categories': None, 'drop': None, 'handle_unknown': 'ignore', 'handle_missing': 'error'}, 'Extra Trees Classifier':{'n_estimators': 100, 'max_features': 'auto', 'max_depth': 6, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_jobs': -1},})

In [15]:
# get the auc and lead scoring score on holdout data
best_pipeline_auc.score(X_holdout, y_holdout,  objectives=["auc", lead_scoring_objective])

OrderedDict([('AUC', 0.6735040797824116),
             ('Lead Scoring', -0.017196904557179708)])

When we optimize for AUC, we can see that the AUC score from this pipeline is better than the AUC score from the pipeline optimized for lead scoring. However, the revenue per lead gained was only `$7` per lead when optimized for AUC and was `$45` when optimized for lead scoring. As a result, we would gain up to 6x the amount of revenue if we optimized for lead scoring.

This happens because optimizing for AUC does not take into account the user-specified true_positive (dollar amount to be gained with a successful lead) and false_positive (dollar amount to be lost with an unsuccessful lead) values. Thus, the best pipelines may produce the highest AUC but may not actually generate the most revenue through lead scoring.

This example highlights how performance in the real world can diverge greatly from machine learning metrics.